In [61]:
import pandas as pd 
import numpy as np
import pandas as pd
import re
import nltk
import spacy
import string
import contractions
from nltk.corpus import stopwords
from collections import Counter
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from nltk.stem.porter import PorterStemmer


In [62]:
fold1 = pd.read_csv("fold1.csv") 
fold2 = pd.read_csv("fold2.csv") 
fold3 = pd.read_csv("fold3.csv") 
fold4 = pd.read_csv("fold4.csv") 
fold5 = pd.read_csv("fold5.csv") 

whole_df = pd.concat([fold1,fold2,fold3,fold4,fold5], ignore_index=True)
lowercased = 0
contractioned = 0
no_punctuation = 0
no_stopwords = 0
no_frequent_words = 0
lemmatization = 0
stemming = 0

In [63]:
#LOWERCASE
def custom_lowercase(text):
    """custom function to lowercase the words that only their first character is capital"""
    new_text = ""
    for word in str(text).split():
        if len(word)>1:
            if word[0].isupper() and word[1].islower():
                word = word[0].lower() + word[1:]
                new_text = new_text +" "+ word
            else:
                new_text = new_text +" "+ word
        elif word[0].isupper():
                word = word[0].lower() 
                new_text = new_text +" "+ word
        else:
            new_text = new_text +" "+ word
    return new_text


whole_df["Review"] = whole_df["Review"].apply(lambda text: custom_lowercase(text))
lowercased = 1

In [64]:
#CONTRACTIONS
whole_df["Review"] = whole_df["Review"].apply(lambda x: contractions.fix(x))
contractioned = 1

In [65]:
#REMOVE PUNCTUATION
#the "0" index of the list is "!" so we skip it
punctuation_list = string.punctuation[1:]
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', punctuation_list))

whole_df["Review"] = whole_df["Review"].apply(lambda text: remove_punctuation(text))
no_punctuation =1

In [66]:
#REMOVE STOPWORDS
#nltk.download('stopwords')
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])
whole_df["Review"] = whole_df["Review"].apply(lambda text: remove_stopwords(text))
no_stopwords = 1

In [67]:
#REMOVE FREQUENT WORDS
#seperate the dataframes
fake_df = whole_df.loc[whole_df['Fake'] == 1]
real_df = whole_df.loc[whole_df['Fake'] == 0]

def counts(df):
    cnt = Counter()
    for text in df["Review"].values:
        for word in text.split():
            cnt[word] += 1
    return cnt

fake_cnts = counts(fake_df)
real_cnts = counts(real_df)
        
def get_list(lis):
    words = []
    for pair in lis:
        words.append(pair[0])
    return words

fake_common = get_list(fake_cnts.most_common(20))
real_common = get_list(real_cnts.most_common(20))
freq_to_remove = list(set(fake_common).intersection(real_common))

def remove_freq(text, freq):
    """custom function to remove freq words"""
    return " ".join([word for word in str(text).split() if word not in freq])

whole_df["Review"] = whole_df["Review"].apply(lambda text: remove_freq(text,freq_to_remove))
no_frequent_words = 1

In [68]:
#LEMMATIZATION
#nltk.download('averaged_perceptron_tagger')

lemmatizer = WordNetLemmatizer()
wordnet_map = {"N":wordnet.NOUN, "V":wordnet.VERB, "J":wordnet.ADJ, "R":wordnet.ADV}
def lemmatize_words(text):
    pos_tagged_text = nltk.pos_tag(text.split())
    return " ".join([lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) for word, pos in pos_tagged_text])

whole_df["Review"] = whole_df["Review"].apply(lambda text: lemmatize_words(text))
lemmatization =1

In [69]:
#STEMMING
stemmer = PorterStemmer()
def stem_words(text):
    return " ".join([stemmer.stem(word) for word in text.split()])

#whole_df["Review"] = whole_df["Review"].apply(lambda text: stem_words(text))
#stemming = 1

In [70]:
X = whole_df['Review']
Y = whole_df['Fake']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, shuffle = False)

uni_vector = TfidfVectorizer(lowercase=False, ngram_range=(1, 1))
features_uni = uni_vector.fit_transform(X_train).toarray()

bi_vector = TfidfVectorizer(lowercase=False, ngram_range=(1, 2))
features_bi = bi_vector.fit_transform(X_train).toarray()

X_train_trans_uni = uni_vector.fit_transform(X_train)
X_test_trans_uni = uni_vector.transform(X_test)

X_train_trans_bi = bi_vector.fit_transform(X_train)
X_test_trans_bi = bi_vector.transform(X_test)

In [71]:
#Building the RLR Model
if (lowercased == 1):
    print("----------")
    print("LOWERCASED")
if (contractioned ==1):
    print("----------")
    print("CONTRACTIONED")
if (no_punctuation == 1):
    print("----------")
    print("NO PUNCTUATION")
if (no_stopwords == 1):
    print("----------")
    print("NO STOPWORDS")
if (no_frequent_words == 1):
    print("----------")
    print("NO FREQ WORDS")
if (lemmatization == 1):
    print("----------")
    print("LEMMATIZATION")
if (stemming == 1):
    print("----------")
    print("STEMMING")
print("----------")

C_param_range = [0.01,0.1,1,10,100]
Penalty = ['l1','l2']

unimax = -1
unibestC = -700
unibestPen = 'none'

bimax = -1
bimax = -1
bibestC = -700
bibestPen = 'none'
for c in C_param_range:
    for p in Penalty:
        print("---------")
        print("C=",c,"  ","P=",p)    
        #unigram
        RLR_uni_model = LogisticRegression(solver='liblinear', penalty=p,C=c)
        RLR_uni_model.fit(X_train_trans_uni, y_train)
        predictions_uni = RLR_uni_model.predict(X_test_trans_uni)
        
        unimetrics = accuracy_score(y_test, predictions_uni) + precision_score(y_test, predictions_uni, zero_division="warn") + recall_score(y_test, predictions_uni, zero_division="warn") + f1_score(y_test, predictions_uni, zero_division="warn")
        
        if(unimetrics>unimax):
            unimax = unimetrics
            unibestC = c
            unibestPen = p         
            
        print("Unigram model")
        print("Accuracy score: ", accuracy_score(y_test, predictions_uni))
        print("Precision score: ", precision_score(y_test, predictions_uni, zero_division="warn"))
        print("Recall score:" , recall_score(y_test, predictions_uni, zero_division="warn"))
        print("F1 score:" , f1_score(y_test, predictions_uni, zero_division="warn"))
        print("Confusion Matrix:", confusion_matrix(y_test, predictions_uni))
        print("")

        #bigram
        RLR_uni_model = LogisticRegression(solver='liblinear', penalty=p,C=c)
        RLR_uni_model.fit(X_train_trans_bi, y_train)
        predictions_bi = RLR_uni_model.predict(X_test_trans_bi)
        bimetrics = accuracy_score(y_test, predictions_bi) + precision_score(y_test, predictions_bi, zero_division="warn") + recall_score(y_test, predictions_bi, zero_division="warn") + f1_score(y_test, predictions_bi, zero_division="warn")
        if(bimetrics>bimax):
            bimax = bimetrics
            bibestC = c
            bibestPen = p
        print("Bigram model")
        print("Accuracy score: ", accuracy_score(y_test, predictions_bi))
        print("Precision score: ", precision_score(y_test, predictions_bi, zero_division="warn"))
        print("Recall score:" , recall_score(y_test, predictions_bi, zero_division="warn"))
        print("F1 score:" , f1_score(y_test, predictions_bi, zero_division="warn"))
        print("Confusion Matrix:", confusion_matrix(y_test, predictions_bi))
        '''if(c==10 and p=='l2'):
            print("---------------")
            print(set(y_test) - set(predictions_uni))
            print(set(y_test) - set(predictions_bi))
            print("---------------")'''
        
print("BEST UNIGRAM has C=",unibestC," and penalty=",unibestPen)
print("BEST BIGRAM has C=",bibestC," and penalty=",bibestPen)

----------
LOWERCASED
----------
CONTRACTIONED
----------
NO PUNCTUATION
----------
NO STOPWORDS
----------
NO FREQ WORDS
----------
LEMMATIZATION
----------
---------
C= 0.01    P= l1
Unigram model
Accuracy score:  0.5
Precision score:  0.0
Recall score: 0.0
F1 score: 0.0
Confusion Matrix: [[80  0]
 [80  0]]

Bigram model
Accuracy score:  0.5
Precision score:  0.0
Recall score: 0.0
F1 score: 0.0
Confusion Matrix: [[80  0]
 [80  0]]
---------
C= 0.01    P= l2
Unigram model
Accuracy score:  0.85
Precision score:  0.8181818181818182
Recall score: 0.9
F1 score: 0.8571428571428572
Confusion Matrix: [[64 16]
 [ 8 72]]

Bigram model
Accuracy score:  0.85
Precision score:  0.8043478260869565
Recall score: 0.925
F1 score: 0.8604651162790697
Confusion Matrix: [[62 18]
 [ 6 74]]
---------
C= 0.1    P= l1
Unigram model
Accuracy score:  0.5
Precision score:  0.0
Recall score: 0.0
F1 score: 0.0
Confusion Matrix: [[80  0]
 [80  0]]

Bigram model
Accuracy score:  0.5
Precision score:  0.0
Recall scor

C:\Users\giann\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy score:  0.86875
Precision score:  0.8933333333333333
Recall score: 0.8375
F1 score: 0.8645161290322579
Confusion Matrix: [[72  8]
 [13 67]]

Bigram model
Accuracy score:  0.88125
Precision score:  0.8860759493670886
Recall score: 0.875
F1 score: 0.8805031446540881
Confusion Matrix: [[71  9]
 [10 70]]
---------
C= 100    P= l1
Unigram model
Accuracy score:  0.7875
Precision score:  0.7948717948717948
Recall score: 0.775
F1 score: 0.7848101265822786
Confusion Matrix: [[64 16]
 [18 62]]

Bigram model
Accuracy score:  0.80625
Precision score:  0.8356164383561644
Recall score: 0.7625
F1 score: 0.7973856209150326
Confusion Matrix: [[68 12]
 [19 61]]
---------
C= 100    P= l2
Unigram model
Accuracy score:  0.85625
Precision score:  0.88
Recall score: 0.825
F1 score: 0.8516129032258064
Confusion Matrix: [[71  9]
 [14 66]]

Bigram model
Accuracy score:  0.875
Precision score:  0.875
Recall score: 0.875
F1 score: 0.875
Confusion Matrix: [[70 10]
 [10 70]]
BEST UNIGRAM has C= 1  and pena